In [1]:
import concurrent.futures as cf
import itertools

import gps2var
import numpy as np
import pyproj
import rasterio

In [2]:
DATA_PATH = "test.tif"

In [3]:
# Write a dummy dataset
profile = {'driver': 'GTiff', 'dtype': 'int8', 'nodata': -128.0, 'width': 36081, 'height': 16382, 'count': 1, 'crs': rasterio.crs.CRS.from_wkt('PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': rasterio.transform.Affine(1000.0, 0.0, -18040094.09752045, 0.0, -1000.0, 9018957.052787267), 'blockxsize': 128, 'blockysize': 64, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}
with rasterio.Env():
    with rasterio.open(DATA_PATH, "w", **profile) as dataset:
        dataset.write(np.zeros(dataset.shape, dtype=dataset.dtypes[0]), 1)

In [4]:
def gen_inputs(scale=1, size=10000, seed=42):
    rng = np.random.default_rng(seed)
    while True:
        yield rng.uniform(-scale, scale, size=(size, 2)).T

## Basic synchronous reading

In [5]:
def time_rasterio_sample(scale):
    """rasterio.DatasetReader.sample()"""
    inputs = gen_inputs(scale=scale)
    with rasterio.open(DATA_PATH) as dataset:
        transformer = pyproj.Transformer.from_crs("EPSG:4326", dataset.crs, always_xy=True)
        %timeit list(dataset.sample(zip(*transformer.transform(*next(inputs)))))

In [6]:
def time_reader(scale):
    """RasterValueReader()"""
    inputs = gen_inputs(scale=scale)
    with gps2var.RasterValueReader(DATA_PATH) as reader:
        %timeit reader.get(*next(inputs))

In [7]:
def time_reader_preload(scale):
    """RasterValueReader(preload_all=True)"""
    inputs = gen_inputs(scale=scale)
    with gps2var.RasterValueReader(DATA_PATH, preload_all=True) as reader:
        %timeit reader.get(*next(inputs))

In [8]:
for scale in [0.1, 1, 10, 45]:
    print(f"scale={scale}")
    for fn in [time_rasterio_sample, time_reader, time_reader_preload]:
        print("  {:40}".format(fn.__doc__), end="")
        fn(scale=scale)

scale=0.1
  rasterio.DatasetReader.sample()         542 ms ± 6.14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
  RasterValueReader()                     8.67 ms ± 63.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
  RasterValueReader(preload_all=True)     7.92 ms ± 64.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
scale=1
  rasterio.DatasetReader.sample()         544 ms ± 4.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
  RasterValueReader()                     9.74 ms ± 65.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
  RasterValueReader(preload_all=True)     8.11 ms ± 45.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
scale=10
  rasterio.DatasetReader.sample()         547 ms ± 10.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
  RasterValueReader()                     68 ms ± 592 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
  RasterValueReader(preload_all=True)     8.46 ms ± 69.2 µs per loop (mean ± 

## Concurrent reading, interpolation

In [9]:
for kwargs in itertools.product([False, True], [1, 10], ["nearest", "bilinear"]):
    kwargs = dict(zip(["preload_all", "scale", "interpolation"], kwargs))
    print(", ".join("{}={!r}".format(k, v) for k, v in kwargs.items()))
    scale = kwargs.pop("scale")

    print("  sync:   ", end="")
    inputs = gen_inputs(scale=scale)
    with gps2var.RasterValueReaderPool(DATA_PATH, num_workers=8, **kwargs) as reader:    
        %timeit [reader.get(*next(inputs)) for _ in range(32)]

    print("  async:  ", end="")
    inputs = gen_inputs(scale=scale)
    with gps2var.RasterValueReaderPool(DATA_PATH, num_workers=8, **kwargs) as reader:
        %timeit list(cf.as_completed(reader.async_get(*next(inputs)) for _ in range(32)))

preload_all=False, scale=1, interpolation='nearest'
  sync:   377 ms ± 3.73 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
  async:  70.4 ms ± 1.87 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
preload_all=False, scale=1, interpolation='bilinear'
  sync:   1.08 s ± 8.82 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
  async:  208 ms ± 4.79 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
preload_all=False, scale=10, interpolation='nearest'
  sync:   2.29 s ± 9.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
  async:  398 ms ± 14.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
preload_all=False, scale=10, interpolation='bilinear'
  sync:   3.68 s ± 23.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
  async:  593 ms ± 13.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
preload_all=True, scale=1, interpolation='nearest'
  sync:   331 ms ± 2.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
  async:  82.5 ms ± 7.91 ms per lo